## Original-Code

In [ ]:
### Listing 1                                                    ###
### implementiert ein einfaches künstliches neuronales Netzwerk. ###
### Feedforward-Netzwerk, das durch Backpropagation              ###
import math
import csv
import numpy as np
from random import shuffle

# Sigmoidfunktion als Aktivierungsfunktion
def sigmoid(x):
    try:
        return 1 / (1 + math.exp(-x))
    except OverflowError:
        return 0

# Künstliches neuronales Netzwerk
class NeuralNetwork:

    # Attribute:
    # - Anzahl Neuronen der Eingabeschicht
    # - Anzahl Neuronen der versteckten Schicht
    # - Anzahl Neuronen der Ausgabeschicht
    # - Lernrate (benannt)
    def __init__(self, i_neurons, h_neurons, o_neurons, learning_rate = 0.1):
        # Grundattribute initialisieren
        self.input_neurons = i_neurons
        self.hidden_neurons = h_neurons
        self.output_neurons = o_neurons
        self.learning_rate = learning_rate
        self.categories = []

        # Gewichte als Zufallswerte initialisieren
        self.input_to_hidden = np.random.rand(
            self.hidden_neurons, self.input_neurons
        ) - 0.5
        #print(type(self.input_to_hidden))
        #print('I to H :',self.input_to_hidden)      # 4 * 12 Matrix
        self.hidden_to_output = np.random.rand(
            self.output_neurons, self.hidden_neurons
        ) - 0.5
        #print('H to O :',self.hidden_to_output)     # 12 * 3 Matrix
        # Aktivierungsfunktion für NumPy-Arrays
        self.activation = np.vectorize(sigmoid)

    # Daten vorbereiten
    # Attribute:
    # data       - Daten als zweidimensionale Liste
    # test_ratio - Anteil, der als Testdaten abgespalten werden soll
    # last       - Kategorie in der letzten Spalte? Sonst in der ersten
    def prepare(self, data, test_ratio=0.1, last=True):
        if last:
            x = [line[0:-1] for line in data]  # Daten/Features
            y = [line[-1] for line in data]    # Kategorie aus letzter Spalte
        else:
            x = [line[1:] for line in data]    # Daten/Features
            y = [line[0] for line in data]     # Kategorie aus erster Spalte
        # Feature-Skalierung (x)
        columns = np.array(x).transpose()      # array erzeugen; Spalte-Zeile tauschen
        #print(np.array(x))
        #print('########\n', columns)
        x_scaled = []
        for column in columns:                 # Normalisieren: max. Wert = 1
            #print(min(column), max(column))   #                min. Wert = 0 
            if min(column) == max(column):
                column = np.zeros(len(column))
            else:
                column = (column - min(column)) / (max(column) - min(column))
            x_scaled.append(column)
        x = np.array(x_scaled).transpose()
        #print(x)
        # Kategorien extrahieren und als Attribut speichern
        y_values = list(set(y))                # Unique durch set()
        self.categories = y_values             # Objektvariable setzen
        # Verteilung auf Ausgabeneuronen (y)
        y_spread = []
        for y_i in y:
            current = np.zeros(len(y_values))  # array 1D, Länge y-Werte mit Nullen
            current[y_values.index(y_i)] = 1   # Setze Index
            y_spread.append(current)           # Array Eintrag in Liste  
        y_out = np.array(y_spread)
        #print(y_out)
        separator = int(test_ratio * len(x))   # Trenner für Testdaten
        return x[:separator], y[:separator], x[separator:], y_out[separator:]

    # Ein einzelner Trainingsdurchgang
    # Attribute:
    # - Eingabedaten als zweidimensionale Liste/Array
    # - Zieldaten als auf Ausgabeneuronen verteilte Liste/Array
    def train(self, inputs, targets):
        # Daten ins richtige Format bringen
        inputs = np.array(inputs, ndmin = 2).transpose()
        targets = np.array(targets, ndmin = 2).transpose()
        # Matrixmultiplikation: Gewichte versteckte Schicht * Eingabe
        hidden_in = np.dot(self.input_to_hidden, inputs)
        # Aktivierungsfunktion anwenden
        hidden_out = self.activation(hidden_in)
        # Matrixmultiplikation: Gewichte Ausgabeschicht * Ergebnis versteckt
        output_in = np.dot(self.hidden_to_output, hidden_out)
        # Aktivierungsfunktion anwenden
        output_out = self.activation(output_in)
        # Die Fehler berechnen
        output_diff = targets - output_out
        hidden_diff = np.dot(self.hidden_to_output.transpose(), output_diff)
        # Die Gewichte mit Lernrate * Fehler anpassen
        self.hidden_to_output += (
            self.learning_rate *
            np.dot(
                (output_diff * output_out * (1.0 - output_out)),
                hidden_out.transpose()
            )
        )
        self.input_to_hidden += (
            self.learning_rate *
            np.dot(
                (hidden_diff * hidden_out * (1.0 * hidden_out)),
                inputs.transpose()
            )
        )

    # Vorhersage für eine Reihe von Testdaten
    # Attribute:
    # - Eingabedaten als zweidimensionale Liste/Array
    # - Vergleichsdaten (benannt, optional)
    def predict(self, inputs, targets = None):
        # Dieselben Schritte wie in train()
        inputs = np.array(inputs, ndmin = 2).transpose()
        hidden_in = np.dot(self.input_to_hidden, inputs)
        hidden_out = self.activation(hidden_in)
        output_in = np.dot(self.hidden_to_output, hidden_out)
        output_out = self.activation(output_in)
        # Ausgabewerte den Kategorien zuweisen
        outputs = output_out.transpose()
        result = []
        for output in outputs:
            result.append(
                self.categories[list(output).index(max(output))]
            )
        # Wenn keine Zielwerte vorhanden, Ergebnisliste zurückgeben
        #print('Ergebnis:',result)
        #print('Vergleich:', targets)
        if targets is None:
            return result
        # Ansonsten vergleichen und korrekte Vorhersagen zählen
        correct = 0
        for res, pred in zip(targets, result):
            if res == pred:
                correct += 1
        percent = correct / len(result) * 100
        return correct, percent


# Hauptprogramm
if __name__ == '__main__':
    with open('iris_nn.csv', 'r') as iris_file:
        reader = csv.reader(iris_file, quoting=csv.QUOTE_NONNUMERIC)
        irises = list(reader)
    shuffle(irises)
    network = NeuralNetwork(4, 12, 3, learning_rate = 0.2)
    x_test, y_test, x_train, y_train = network.prepare(irises, test_ratio=0.2)
    #print(x_train,'\n',y_train)
    #print(x_test,'\n',y_test)
    for i in range(200):
        network.train(x_train, y_train)
    correct, percent = network.predict(x_test, targets = y_test)
    print(f"{correct} korrekte Vorhersagen ({percent}%).")


## Hinzufügen einer variablen Anzahl von hidden Layers

In [16]:
### Listing 1                                                    ###
### implementiert ein einfaches künstliches neuronales Netzwerk. ###
### Feedforward-Netzwerk, das durch Backpropagation              ###
import math
import csv
import numpy as np
from random import shuffle

# Sigmoidfunktion als Aktivierungsfunktion
def sigmoid(x):
    try:
        return 1 / (1 + math.exp(-x))
    except OverflowError:
        return 0

# Künstliches neuronales Netzwerk
class NeuralNetwork:

    # Attribute:
    # - Anzahl Neuronen der Eingabeschicht
    # - Anzahl Neuronen der versteckten Schicht
    # - Anzahl Neuronen der Ausgabeschicht
    # - Lernrate (benannt)
    def __init__(self, i_neurons, h_neurons, o_neurons, learning_rate = 0.1):
        # Grundattribute initialisieren
        self.input_neurons = i_neurons
        
#         h_neurons is now passed as a list with multiple values
#         indicating the number of neurons inside the different hidden layers
        self.hidden_neurons = h_neurons
        self.output_neurons = o_neurons
        self.learning_rate = learning_rate
        self.categories = []

        # Gewichte als Zufallswerte initialisieren
        self.weights = {}
        self.weights["input_to_hidden1"] = np.random.rand(
            self.hidden_neurons[0], self.input_neurons
        ) - 0.5
        
        if len(self.hidden_neurons) > 1:
            for i, _ in enumerate(self.hidden_neurons[1:]):
                self.weights[f"hidden{i+1}_to_hidden{i+2}"] = np.random.rand(
                    self.hidden_neurons[i+1] , self.hidden_neurons[i]
                ) - 0.5


        self.weights[f"hidden{len(self.hidden_neurons)}_to_output"] = np.random.rand(
            self.output_neurons, self.hidden_neurons[-1]
        ) - 0.5

        # Aktivierungsfunktion für NumPy-Arrays
        self.activation = np.vectorize(sigmoid)

    # Daten vorbereiten
    # Attribute:
    # data       - Daten als zweidimensionale Liste
    # test_ratio - Anteil, der als Testdaten abgespalten werden soll
    # last       - Kategorie in der letzten Spalte? Sonst in der ersten
    def prepare(self, data, test_ratio=0.1, last=True):
        if last:
            x = [line[0:-1] for line in data]  # Daten/Features
            y = [line[-1] for line in data]    # Kategorie aus letzter Spalte
        else:
            x = [line[1:] for line in data]    # Daten/Features
            y = [line[0] for line in data]     # Kategorie aus erster Spalte
        # Feature-Skalierung (x)
        columns = np.array(x).transpose()      # array erzeugen; Spalte-Zeile tauschen
        #print(np.array(x))
        #print('########\n', columns)
        x_scaled = []
        for column in columns:                 # Normalisieren: max. Wert = 1
            #print(min(column), max(column))   #                min. Wert = 0 
            if min(column) == max(column):
                column = np.zeros(len(column))
            else:
                column = (column - min(column)) / (max(column) - min(column))
            x_scaled.append(column)
        x = np.array(x_scaled).transpose()
        #print(x)
        # Kategorien extrahieren und als Attribut speichern
        y_values = list(set(y))                # Unique durch set()
        self.categories = y_values             # Objektvariable setzen
        # Verteilung auf Ausgabeneuronen (y)
        y_spread = []
        for y_i in y:
            current = np.zeros(len(y_values))  # array 1D, Länge y-Werte mit Nullen
            current[y_values.index(y_i)] = 1   # Setze Index
            y_spread.append(current)           # Array Eintrag in Liste  
        y_out = np.array(y_spread)
        #print(y_out)
        separator = int(test_ratio * len(x))   # Trenner für Testdaten
        return x[:separator], y[:separator], x[separator:], y_out[separator:]

    # Ein einzelner Trainingsdurchgang
    # Attribute:
    # - Eingabedaten als zweidimensionale Liste/Array
    # - Zieldaten als auf Ausgabeneuronen verteilte Liste/Array
    def train(self, inputs, targets):
        # Daten ins richtige Format bringen
        inputs = np.array(inputs, ndmin = 2).transpose()
        targets = np.array(targets, ndmin = 2).transpose()
        # Matrixmultiplikation: Gewichte versteckte Schicht * Eingabe
        
        self.training = {}
        
        self.training["hidden1_in"] = np.dot(self.weights["input_to_hidden1"], inputs)
        # Aktivierungsfunktion anwenden
        self.training["hidden1_out"] = self.activation(self.training["hidden1_in"])
        
        if len(self.hidden_neurons) > 1:
            for i, _ in enumerate(self.hidden_neurons[1:]):
                self.training[f"hidden{i+2}_in"]  = np.dot(self.weights[f"hidden{i+1}_to_hidden{i+2}"], self.training["hidden1_out"])
                self.training[f"hidden{i+2}_out"] = self.activation(self.training[f"hidden{i+2}_in"])
        
        # Matrixmultiplikation: Gewichte Ausgabeschicht * Ergebnis versteckt
        output_in = np.dot(self.weights[f"hidden{len(self.hidden_neurons)}_to_output"], self.training[f"hidden{len(self.hidden_neurons)}_out"])
        # Aktivierungsfunktion anwenden
        output_out = self.activation(output_in)
        
        # Die Fehler berechnen
        self.errors = {}
        output_diff = targets - output_out
        
        self.errors[f"hidden{len(self.hidden_neurons)}_diff"] = np.dot(self.weights[f"hidden{len(self.hidden_neurons)}_to_output"].transpose(), output_diff)
        
        if len(self.hidden_neurons) > 1:
            for i in range(len(self.hidden_neurons)-1)[::-1]:
                self.errors[f"hidden{i+1}_diff"] = np.dot(self.weights[f"hidden{i+1}_to_hidden{i+2}"].transpose(), self.errors[f"hidden{len(self.hidden_neurons)}_diff"])   
        
        # Die Gewichte mit Lernrate * Fehler anpassen
        self.weights[f"hidden{len(self.hidden_neurons)}_to_output"] += (
            self.learning_rate *
            np.dot(
                (output_diff * output_out * (1.0 - output_out)),
                self.training[f"hidden{len(self.hidden_neurons)}_out"].transpose()
            )
        )
        
        if len(self.hidden_neurons) > 1:
            for i, _ in enumerate(self.hidden_neurons[1:]):
                self.weights[f"hidden{i+1}_to_hidden{i+2}"] += (
                    self.learning_rate *
                    np.dot(
                        (self.errors[f"hidden{i+2}_diff"] * self.training[f"hidden{i+2}_out"] * (1.0 - self.training[f"hidden{i+2}_out"])),
                        self.training[f"hidden{i+1}_out"].transpose()
                    )
                )
        
        self.weights["input_to_hidden1"] += (
            self.learning_rate *
            np.dot(
                (self.errors["hidden1_diff"] * self.training["hidden1_out"] * (1.0 - self.training["hidden1_out"])),
                inputs.transpose()
            )
        )

    # Vorhersage für eine Reihe von Testdaten
    # Attribute:
    # - Eingabedaten als zweidimensionale Liste/Array
    # - Vergleichsdaten (benannt, optional)
    def predict(self, inputs, targets = None):
        # Dieselben Schritte wie in train()
        inputs = np.array(inputs, ndmin = 2).transpose()
        self.training["hidden1_in"] = np.dot(self.weights["input_to_hidden1"], inputs)
        # Aktivierungsfunktion anwenden
        self.training["hidden1_out"] = self.activation(self.training["hidden1_in"])
        
        if len(self.hidden_neurons) > 1:
            for i, _ in enumerate(self.hidden_neurons[1:]):
                self.training[f"hidden{i+2}_in"]  = np.dot(self.weights[f"hidden{i+1}_to_hidden{i+2}"], self.training["hidden1_out"])
                self.training[f"hidden{i+2}_out"] = self.activation(self.training[f"hidden{i+2}_in"])
        
        # Matrixmultiplikation: Gewichte Ausgabeschicht * Ergebnis versteckt
        output_in = np.dot(self.weights[f"hidden{len(self.hidden_neurons)}_to_output"], self.training[f"hidden{len(self.hidden_neurons)}_out"])
        # Aktivierungsfunktion anwenden
        output_out = self.activation(output_in)

        # Ausgabewerte den Kategorien zuweisen
        outputs = output_out.transpose()
        result = []
        for output in outputs:
            result.append(
                self.categories[list(output).index(max(output))]
            )
        # Wenn keine Zielwerte vorhanden, Ergebnisliste zurückgeben
        #print('Ergebnis:',result)
        #print('Vergleich:', targets)
        if targets is None:
            return result
        # Ansonsten vergleichen und korrekte Vorhersagen zählen
        correct = 0
        for res, pred in zip(targets, result):
            if res == pred:
                correct += 1
        percent = correct / len(result) * 100
        return correct, percent


def training_result(data, h_neurons = [6], learning_param = 0.03, repeats = 100):
    # Hauptprogramm
        network = NeuralNetwork(4, h_neurons, 3, learning_rate = learning_param)
        x_test, y_test, x_train, y_train = network.prepare(data, test_ratio=0.2)
        for i in range(repeats):
            network.train(x_train, y_train)
        correct, percent = network.predict(x_test, targets = y_test)
        return percent


In [20]:
def get_data():
    if __name__ == '__main__':
        with open('iris_nn.csv', 'r') as iris_file:
            reader = csv.reader(iris_file, quoting=csv.QUOTE_NONNUMERIC)
            irises = list(reader)
        shuffle(irises)
        return irises

## Funktion für automatisches Tuning

In [14]:
# from timeit import timeit as ti
import numpy as np
import pandas as pd
from statistics import median, mean_arith, variance
# from tqdm import tqdm
from math import floor, ceil

In [58]:
data = [ get_data() for i in range(20) ]
data = [ (round(training_result(d)),d) for d in data ]
data = sorted(data)[len(data)//2][1]
# data = get_data()

h_neurons = [[(0,6)]]
learning_rate = [(0,0.03)]
repeats = [(0,100)]

params = [*h_neurons, learning_rate, repeats]

factor = 1.2
list_size = 30
for i in range(10):
    results = []
    for j,param in enumerate(params):
        
        one_changed_param = [*(list(zip(*params))[-1])]
        if j != len(h_neurons):
            param_increase = ceil(param[-1][1]*factor)
            param_decrease = floor(param[-1][1]/factor)
        else:
            param_increase = round(param[-1][1]*factor,5)
            param_decrease = round(param[-1][1]/factor,5)
            
        one_changed_param[j] = (0, param_increase)
        increase_variance = variance([training_result(data, [ l[1] for l in one_changed_param[:len(h_neurons)] ], one_changed_param[-2][1], one_changed_param[-1][1]) for k in range(list_size)])
        one_changed_param[j] = (0, param_decrease)
        decrease_variance = variance([training_result(data, [ l[1] for l in one_changed_param[:len(h_neurons)] ], one_changed_param[-2][1], one_changed_param[-1][1]) for k in range(list_size)])
        
        if increase_variance < decrease_variance:
            results.append((increase_variance, param_increase))
        else:
            results.append((decrease_variance, param_decrease))
    
    for k,result in enumerate(results):
        params[k].append(result)
        
    min_variance = min([k for k,l in results])
    
    for k,result in enumerate(results):
        if min_variance == result[0]:
            params[k].append(result)
        else:
            params[k].append(params[k][-2])
            
df = pd.DataFrame()

for i in range(len(h_neurons)):
    df[f"neurons_{i+1}_value"] = [ k for j,k in params[i]]
    df[f"neurons_{i+1}_change"] = [ j for j,k in params[i]]

df["learning_rate_value"] = [ k for j,k in params[-2]]
df["learning_rate_change"] = [ j for j,k in params[-2]]
df["repeats_value"] = [ k for j,k in params[-1]]
df["repeats_change"] = [ j for j,k in params[-1]]

df

,neurons_1_value,neurons_1_change,learning_rate_value,learning_rate_change,repeats_value,repeats_change
0,6,0.000000,0.0300,0.000000,100,0.000000
1,5,6.245211,0.0360,10.076628,120,9.310345
2,5,6.245211,0.0300,0.000000,100,0.000000
3,6,9.450830,0.0360,6.500639,120,7.918263
4,5,6.245211,0.0360,6.500639,100,0.000000
5,6,6.500639,0.0432,5.734355,120,4.865900
6,5,6.245211,0.0360,6.500639,120,4.865900
7,4,6.436782,0.0300,5.363985,100,5.312899
8,5,6.245211,0.0360,6.500639,100,5.312899
9,6,8.033206,0.0432,5.619413,120,6.130268
